In [ ]:
import pandas as pd
import seaborn as sns

from utils.tags import tag_elements

In [ ]:
# 'https://drive.google.com/uc?id=1_osI1ZMtR3chrKtwRfwlNyrr5VP0OIT0'

article_data = pd.read_csv('./datasets/article_about_data.gz',
                           compression='gzip',
                           parse_dates=['datetime'],
                           index_col='id')

article_data

__Предобработка данных__

_Предобработка тэгов_

In [ ]:
# Теги загружены в виде строки вида ['tag_1', 'tag_1', 'tag_1'],
# привожу его к массиву.

article_data.tags = [tag for tag in article_data.tags.str.replace(
                     pat="[\[\]']", repl='', regex=True).str.split(',') if tag != '']


In [ ]:
# Раскрываю датафрейм по колонке tags. Применяю к каждому отдельному тэгу tag_elements,
# которая очищает тэг и разделяет его на отдельные слова (массив слов).
# Повторно раскрываю датафрейм по колонке tags.

article_data = article_data.explode('tags')
article_data.tags = article_data.tags.apply(tag_elements)
article_data = article_data.explode('tags')

article_data

In [ ]:
# Выбираю тэги и подсчитываю их встречаемость.
# Рассчитываю долю встречаемости конкретного тэга.

tags = article_data.groupby('tags', as_index=False) \
                   .agg({'comments': 'count'}) \
                   .rename(columns={'tags': 'tag', 'comments': 'used'}) \
                   .sort_values(['used', 'tag'])

# Удаляю тэги, использованные только один раз
tags.drop(tags.query('used == 1').index, inplace=True)

# Определяю язык тэга
tags = tags.assign(ru = tags.tag.str.contains('^[а-я]+$', regex=True),
                   en = tags.tag.str.contains('^[a-z]+$', regex=True))

# Удаляю теги на двух языках
tags.drop(tags.query('ru == False and en == False').index, inplace=True)

tags

In [ ]:
# Какая-то полезная работа с тэгами...

In [ ]:
# Совмещаю данные по тэгам с исходным датафреймом.

article_data = article_data.merge(tags, left_on='tags', right_on='tag', how='left') \
                           .drop(columns='tags') \
                           .dropna()

article_data

__Это на потом, может быть использую.__

In [ ]:
# 'https://drive.google.com/uc?id=1YjYmulVuJPH0IP770enNPWo9WPHdf1U0'

# article_text = pd.read_csv('./datasets/article_text_data.gz',
#                            compression='gzip',
#                            index_col='id')
